# MegaDescriptor-S-224
- Run inference with MegaDescriptor-S-224 (https://huggingface.co/BVRA/MegaDescriptor-S-224)

In [3]:
import pandas as pd
from torchvision import transforms as T
from timm import create_model

from wildlife_tools.features import DeepFeatures
from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier


datasets = [
    'BirdIndividualID',
    'SealID',
    'FriesianCattle2015',
    'ATRW',
    'NDD20',
    'SMALST',
    'SeaTurtleIDHeads',
    'AAUZebraFish',
    'CZoo',
    'CTai',
    'Giraffes',
    'HyenaID2022',
    'MacaqueFaces',
    'OpenCows2020',
    'StripeSpotter',
    'AerialCattle2017',
    'GiraffeZebraID',
    'IPanda50',
    'WhaleSharkID',
    'FriesianCattle2017',
    'Cows2021',
    'LeopardID2022',
    'NOAARightWhale',
    'HappyWhale',
    'HumpbackWhaleID',
    'LionData',
    'NyalaData',
    'ZindiTurtleRecall',
    'BelugaID',
    ]

model = create_model("hf-hub:BVRA/MegaDescriptor-S-224", pretrained=True)
extractor = DeepFeatures(model, device='cuda')

root_images = '../data/images/size-256'
root_metadata = '../data/metadata/datasets'

In [2]:
results = {}
for name in datasets:
    metadata = pd.read_csv(f'{root_metadata}/{name}/metadata.csv', index_col=0)

    transform = T.Compose([
        T.Resize(size=(224, 224)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    database = WildlifeDataset(
        metadata=metadata,
        root=f'{root_images}/{name}/',
        transform=transform,
        split=SplitMetadata('split', 'train'),
    )

    query = WildlifeDataset(
        metadata=metadata,
        root=f'{root_images}/{name}/',
        transform=transform,
        split=SplitMetadata('split', 'test'),
    )

    matcher = CosineSimilarity()
    similarity = matcher(query=extractor(query), database=extractor(database))
    preds = KnnClassifier(k=1, database_labels=database.labels_string)(similarity['cosine'])
    
    acc = sum(preds == query.labels_string) / len(preds)
    print(name, acc)
    results[name] = acc


pd.Series(results).to_csv('results/MegaDescriptor-S-224.csv')

100%|█████████████████████████████████████████████████████████████| 327/327 [11:24<00:00,  2.09s/it]


BirdIndividualID 0.9680566182096404


100%|███████████████████████████████████████████████████████████████| 13/13 [00:27<00:00,  2.11s/it]


SealID 0.6330935251798561


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.19s/it]


...

NyalaData 0.14953271028037382


100%|███████████████████████████████████████████████████████████████| 76/76 [01:34<00:00,  1.24s/it]


ZindiTurtleRecall 0.3838416612589228


100%|███████████████████████████████████████████████████████████████| 37/37 [01:07<00:00,  1.82s/it]


BelugaID 0.3884103367267032
